## Example notebook

In [5]:
from bnsl.algorithms.silander_myllymaki import run as silander_myllymaki
from bnsl.algorithms.partial_order_approach import run as partial_order_approach
from bnsl.algorithms.approximation_algorithm import run as approximation_algorithm
from bnsl.transforms.shifts import get_shift, get_upper_bound
from bnsl.scoring import read_local_scores, write_local_scores
from bnsl.sampling import sample_data

In [8]:
NETWORK_PATH = "../../networks/small/asia.bif"

### Sample data

In [9]:
dat = sample_data(NETWORK_PATH, n_samples=1000, seed=42)

  0%|          | 0/8 [00:00<?, ?it/s]

### Write local scores to file

In [10]:
jaa = write_local_scores(dat)

Restricted license - for non-production use only - expires 2025-11-24


INFO:gurobipy:Restricted license - for non-production use only - expires 2025-11-24


Set parameter PreCrush to value 1


INFO:gurobipy:Set parameter PreCrush to value 1


Set parameter CutPasses to value 100000


...

INFO:gurobipy:Set parameter ZeroHalfCuts to value 2


Set parameter MIPGap to value 0


INFO:gurobipy:Set parameter MIPGap to value 0


Set parameter MIPGapAbs to value 0


INFO:gurobipy:Set parameter MIPGapAbs to value 0


### Helper function

In [12]:
def print_result(method_name: str, result):
    print(f"[{method_name}] score={result.total_score:.3f}")
    print(f"[{method_name}] parent map")
    for v in result.pm:
        print(f"  {v}: {result.pm[v]}")

### Sylander Myllymaki

In [13]:
result_dp = silander_myllymaki(jaa)
print_result("Silander & Myllymaki", result_dp)

[Silander & Myllymaki] score=-2235.024
[Silander & Myllymaki] parent map
  lung: frozenset()
  tub: frozenset()
  either: frozenset({'tub', 'lung'})
  xray: frozenset({'either'})
  smoke: frozenset({'lung'})
  asia: frozenset()
  bronc: frozenset({'smoke'})
  dysp: frozenset({'bronc', 'either'})


### Partial order approach:

In [14]:
### Arguments for partial order approach ###
m:int = 2
p:int = 2

In [15]:
result_poa = partial_order_approach(jaa, m=m, p=p)
print_result("Partial Order Approach", result_poa)

[partial_order_approach] Total partial orders to evaluate: 4
[Partial Order Approach] score=-2235.024
[Partial Order Approach] parent map
  asia: frozenset()
  bronc: frozenset({'smoke'})
  dysp: frozenset({'bronc', 'either'})
  either: frozenset({'tub', 'lung'})
  lung: frozenset({'smoke'})
  smoke: frozenset()
  tub: frozenset()
  xray: frozenset({'either'})


### Approximation algorithm:

In [16]:
### Arguments for approximation algorithm ### 
l:int = 2
k:int = 3

In [17]:
result_approx = approximation_algorithm(jaa, l=l, k=k)
print_result("Approximation Algorithm", result_approx)

[Approximation Algorithm] score=-2238.429
[Approximation Algorithm] parent map
  asia: frozenset()
  bronc: frozenset({'smoke'})
  dysp: frozenset({'bronc', 'either'})
  either: frozenset({'lung'})
  lung: frozenset({'smoke'})
  smoke: frozenset()
  tub: frozenset({'lung', 'either'})
  xray: frozenset({'either'})


### Find the theoretical UB for the approximation algorithm

In [19]:
LS = read_local_scores(jaa)
shift = get_shift(LS)
n = len(LS)
ub = get_upper_bound(shift, n, result_approx.total_score, l / k)

In [20]:
print(f"Approximation Algorithm Upper Bound: {ub:.3f}")
print(f"Approximation Score: {result_approx.total_score:.3f}")
print(f"True optimal score (DP): {result_dp.total_score:.3f}")

Approximation Algorithm Upper Bound: -3348.523
Approximation Score: -2238.429
True optimal score (DP): -2235.024
